In [29]:
import pandas as pd
from sklearn.preprocessing import label_binarize
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation

import warnings
warnings.filterwarnings("ignore")

In [30]:
import os

In [31]:
os.getcwd()

'/Users/rajatsuri/Desktop/Padhai/CIS 508/assignment2'

In [32]:
#define working directory
os.chdir('/Users/rajatsuri/Desktop/Padhai/CIS 508/assignment2')

In [33]:
os.getcwd()

'/Users/rajatsuri/Desktop/Padhai/CIS 508/assignment2'

In [139]:
Train = pd.read_csv ('Portugese Bank Data - TRAIN.csv')
Test = pd.read_csv('Portugese Bank Data - TEST.csv')
Test1 = pd.read_excel('Portugese Bank Data - TEST(1).xlsx')

In [140]:
Train.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [141]:
Test.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [142]:
Train.shape

(4521, 17)

In [143]:
Train.dtypes

age           int64
job          object
marital      object
education    object
default      object
balance       int64
housing      object
loan         object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [144]:
Train.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [146]:
Train.describe()

,age,balance,day,duration,campaign,pdays,previous
count,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000,4521.000000
mean,41.170095,1422.657819,15.915284,263.961292,2.793630,39.766645,0.542579
std,10.576211,3009.638142,8.247667,259.856633,3.109807,100.121124,1.693562
min,19.000000,-3313.000000,1.000000,4.000000,1.000000,-1.000000,0.000000
25%,33.000000,69.000000,9.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,444.000000,16.000000,185.000000,2.000000,-1.000000,0.000000
75%,49.000000,1480.000000,21.000000,329.000000,3.000000,-1.000000,0.000000
max,87.000000,71188.000000,31.000000,3025.000000,50.000000,871.000000,25.000000


In [147]:
# seperating input data and data we want to predict
inputs = Train.drop('y', axis='columns')
target = Train['y']


In [148]:
#list of categorical features that need onehot encoding 
categoricalFeatures =['job', 'marital', 'education', 'default', 'housing', 'loan', 
                       'contact', 'month', 'poutcome']

In [149]:
ohe = OneHotEncoder(handle_unknown='ignore',sparse=False)
Xcat = pd.DataFrame(ohe.fit_transform(inputs[categoricalFeatures]),columns=ohe.get_feature_names(),index=inputs.index)

In [150]:
inputs = pd.concat([inputs,Xcat],axis=1)

In [151]:
inputs.drop(labels=categoricalFeatures,axis=1,inplace=True)

In [152]:
inputs.sample(5)

,age,balance,day,duration,campaign,pdays,previous,x0_admin.,x0_blue-collar,x0_entrepreneur,...,x7_jun,x7_mar,x7_may,x7_nov,x7_oct,x7_sep,x8_failure,x8_other,x8_success,x8_unknown
3811,26,-402,12,127,2,-1,0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2385,47,3333,29,114,2,-1,0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4482,46,466,29,169,1,-1,0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4392,50,3357,9,239,4,-1,0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
942,36,222,6,317,1,169,2,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [153]:
Test.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [154]:
Xcat = pd.DataFrame(ohe.transform(Test[categoricalFeatures]),columns=ohe.get_feature_names(),index=Test.index)

In [155]:
# OneHotEncoding on Test (only transform)
# OneHotEncoding is to be done on Categorical variables.
Xcat = pd.DataFrame(ohe.transform(Test[categoricalFeatures]),columns=ohe.get_feature_names(),index=Test.index)
Test = pd.concat([Test,Xcat],axis=1)
Test.drop(labels=categoricalFeatures,axis=1,inplace=True)
Test.sample(5)

,age,balance,day,duration,campaign,pdays,previous,y,x0_admin.,x0_blue-collar,...,x7_jun,x7_mar,x7_may,x7_nov,x7_oct,x7_sep,x8_failure,x8_other,x8_success,x8_unknown
31038,26,1222,11,108,1,-1,0,no,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
14500,29,2312,14,109,3,-1,0,no,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
7541,38,5669,30,31,1,-1,0,no,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24298,55,2410,17,135,1,-1,0,no,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
9393,43,101,6,153,1,-1,0,no,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [156]:
from sklearn import tree
from sklearn import set_config

In [157]:
#Decision Tree Model
modelDF = tree.DecisionTreeClassifier()

In [158]:
#Fitting the model on train data
DF = modelDF.fit(inputs,target)

In [159]:
print(modelDF.fit(inputs,target))
set_config(print_changed_only=True)


DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')


In [160]:
rf = RandomForestClassifier()

In [161]:
rf.fit(inputs, target)

RandomForestClassifier()

In [163]:
print(rf.fit(inputs, target))
set_config(print_changed_only=True)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)


In [164]:
# seperating input data and data we want to predict
Xtest = Test.drop('y', axis='columns')
Ytest = Test['y']

In [165]:
X_Pred = modelDF.predict(Xtest)
XPred = modelDF.predict(inputs)
#Model Accuracy
print("Train Accuracy:", metrics.accuracy_score(target,XPred))
print("Test Accuracy:", metrics.accuracy_score(Ytest,X_Pred))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,X_Pred))
print("Max Depth",modelDF.get_depth())
print("Leaf",modelDF.get_n_leaves())
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, X_Pred))

Train Accuracy: 1.0
Test Accuracy: 0.8817986773130433
Confusion Matrix for Decision Tree:
[[37162  2760]
 [ 2584  2705]]
Max Depth 26
Leaf 385
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.93      0.93      0.93     39922
         yes       0.49      0.51      0.50      5289

    accuracy                           0.88     45211
   macro avg       0.71      0.72      0.72     45211
weighted avg       0.88      0.88      0.88     45211



In [166]:
X_Pred1 = rf.predict(Xtest)
XPred1 = rf.predict(inputs)
#Model Accuracy
print("Train Accuracy:", metrics.accuracy_score(target,XPred1))
print("Test Accuracy:", metrics.accuracy_score(Ytest,X_Pred1))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,X_Pred1))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, X_Pred1))

Train Accuracy: 1.0
Test Accuracy: 0.9129194222644932
Confusion Matrix for Decision Tree:
[[39179   743]
 [ 3194  2095]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.92      0.98      0.95     39922
         yes       0.74      0.40      0.52      5289

    accuracy                           0.91     45211
   macro avg       0.83      0.69      0.73     45211
weighted avg       0.90      0.91      0.90     45211



In [291]:
#Hyperparameter tuning done for decision tree classifier-Marketing

#RANDOM SEARCH--------------------------------------------

import time
start_time = time.time()

print("RandomizedSearchCV-Decision tree")
parameters={'min_samples_leaf' : range(10,31,6),'max_depth': 
            range(26,51,17),'criterion':['gini']}
dt_random = RandomizedSearchCV(modelDF,parameters,n_iter=25,cv=5)
dt_random.fit(inputs, target)
grid_parm=dt_random.best_params_
print(grid_parm)
print("accuracy Score for Decision Tree:{0:6f}".
      format(dt_random.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

RandomizedSearchCV-Decision tree
{'min_samples_leaf': 22, 'max_depth': 43, 'criterion': 'gini'}
accuracy Score for Decision Tree:0.899914
--- 1.766937017440796 seconds ---


In [216]:
#GRID SEARCH----------------------------------------

import time
start_time = time.time()

print("GridSearchCV-Decision tree")
dt_grid = GridSearchCV(modelDF,parameters)
dt_grid.fit(inputs, target)
grid_parm1=dt_grid.best_params_
print(grid_parm1)
print("accuracy Score for Decision Tree:{0:6f}".
      format(dt_grid.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

GridSearchCV-Decision tree
{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 60}
accuracy Score for Decision Tree:0.896596
--- 64.58353304862976 seconds ---


In [217]:
#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
dtRand = DecisionTreeClassifier(**grid_parm)
dtGrid = DecisionTreeClassifier(**grid_parm1)

dtRand.fit(inputs,target)
dtRand_predict = dtRand.predict(Xtest)
dtGrid.fit(inputs,target)
dtGrid_predict = dtGrid.predict(Xtest)

In [218]:
# Accuracy for Decision Tree using Random Search CV for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,dtRand_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,dtRand_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, dtRand_predict))
clf_cv_score = cross_val_score(dtRand, inputs, target, cv=5, scoring="balanced_accuracy")
print(clf_cv_score)

Test Accuracy: 0.895025546880184
Confusion Matrix for Decision Tree:
[[38578  1344]
 [ 3402  1887]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.92      0.97      0.94     39922
         yes       0.58      0.36      0.44      5289

    accuracy                           0.90     45211
   macro avg       0.75      0.66      0.69     45211
weighted avg       0.88      0.90      0.88     45211

[0.64401786 0.63509615 0.65139423 0.68192308 0.64197115]


In [219]:
# Accuracy for Decision Tree using Grid Search for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,dtGrid_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,dtGrid_predict))
print('Printing the precision and recall, among other metrics')

Test Accuracy: 0.8965959611598947
Confusion Matrix for Decision Tree:
[[38265  1657]
 [ 3018  2271]]
Printing the precision and recall, among other metrics


In [220]:
#Hyperparameter tuning done for random forest classifier-Marketing

#RANDOM SEARCH--------------------------------------------

import time
start_time = time.time()

print("RandomizedSearchCV-Random forest")
rand_parameters={'min_samples_leaf' : range(10,100,10),'max_depth': 
            range(1,10,2),'max_features':[10,20,30],'n_estimators':[20,30,40]}
rf_random = RandomizedSearchCV(rf,rand_parameters,n_iter=25,cv=5)
rf_random.fit(inputs, target)
grid_parm=rf_random.best_params_
print(grid_parm)
print("accuracy Score for Decision Tree:{0:6f}".
      format(rf_random.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

RandomizedSearchCV-Random forest
{'n_estimators': 30, 'min_samples_leaf': 10, 'max_features': 30, 'max_depth': 5}
accuracy Score for Decision Tree:0.903099
--- 21.051417112350464 seconds ---


In [221]:
import time
start_time = time.time()

print("GridSearchCV-Random Forest")
rf_grid = GridSearchCV(rf,rand_parameters)
rf_grid.fit(inputs, target)
grid_parm1=rf_grid.best_params_
print(grid_parm1)
print("accuracy Score for Decision Tree:{0:6f}".
      format(rf_grid.score(Xtest,Ytest)))

print("--- %s seconds ---" % (time.time() - start_time))

GridSearchCV-Random Forest
{'max_depth': 5, 'max_features': 20, 'min_samples_leaf': 10, 'n_estimators': 30}
accuracy Score for Decision Tree:0.901971
--- 288.44663286209106 seconds ---


In [222]:
#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier 
rfRand = RandomForestClassifier(**grid_parm)
rfGrid = RandomForestClassifier(**grid_parm1)

rfRand.fit(inputs,target)
rfRand_predict = rfRand.predict(Xtest)
rfGrid.fit(inputs,target)
rfGrid_predict = rfGrid.predict(Xtest)

In [223]:
# Accuracy for Random Forest using Random Search CV for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,rfRand_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,rfRand_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, rfRand_predict))
clf_cv_score = cross_val_score(rfRand, inputs, target, cv=5, scoring="balanced_accuracy")
print(clf_cv_score)

Test Accuracy: 0.9015726261308089
Confusion Matrix for Decision Tree:
[[39004   918]
 [ 3532  1757]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.92      0.98      0.95     39922
         yes       0.66      0.33      0.44      5289

    accuracy                           0.90     45211
   macro avg       0.79      0.65      0.69     45211
weighted avg       0.89      0.90      0.89     45211

[0.62973214 0.64971154 0.62442308 0.63004808 0.64384615]


In [224]:
# Accuracy for Random Forest using Grid Search for Hyperparameter Tuning

print("Test Accuracy:", metrics.accuracy_score(Ytest,rfGrid_predict))
print("Confusion Matrix for Decision Tree:")
print(confusion_matrix(Ytest,rfGrid_predict))
print('Printing the precision and recall, among other metrics')
print(metrics.classification_report(Ytest, rfGrid_predict))

Test Accuracy: 0.9013514410209905
Confusion Matrix for Decision Tree:
[[39195   727]
 [ 3733  1556]]
Printing the precision and recall, among other metrics
              precision    recall  f1-score   support

          no       0.91      0.98      0.95     39922
         yes       0.68      0.29      0.41      5289

    accuracy                           0.90     45211
   macro avg       0.80      0.64      0.68     45211
weighted avg       0.89      0.90      0.88     45211

